In [16]:
import pandas as pd
import numpy as np

In [17]:
data_set = pd.read_csv('../dataset/most_viewed_youtube_shorts.csv')

In [18]:
data_set.head()

,title,published_at,duration,view_count,like_count,comment_count
0,"She Was ""One Minute"" Away From Disaster! #shorts",2023-02-11T16:39:01Z,PT59S,746717484,13068287.0,19863.0
1,I HAVE BANGS… #Shorts,2021-08-30T09:42:50Z,PT23S,555825641,16003237.0,NaN
2,funny vadieo subscribe for more vadieos #shorts,2023-05-25T08:00:01Z,PT28S,482962560,8355647.0,15416.0
3,She turned pregnant and he let her go! #shorts,2022-07-25T11:00:43Z,PT1M,415148935,3954689.0,3807.0
4,Tom&Jerry.. Real End Twist 😆#shorts,2023-06-13T13:40:23Z,PT1M,395791994,6965246.0,NaN


# DATA CLEANING

In [19]:
#Dupilcate Rows  
data_set[data_set['title'].duplicated(keep=False)].count()

title            65
published_at     65
duration         65
view_count       65
like_count       63
comment_count    58
dtype: int64

In [20]:
data_set.drop_duplicates(keep='first')

,title,published_at,duration,view_count,like_count,comment_count
0,"She Was ""One Minute"" Away From Disaster! #shorts",2023-02-11T16:39:01Z,PT59S,746717484,13068287.0,19863.0
1,I HAVE BANGS… #Shorts,2021-08-30T09:42:50Z,PT23S,555825641,16003237.0,NaN
2,funny vadieo subscribe for more vadieos #shorts,2023-05-25T08:00:01Z,PT28S,482962560,8355647.0,15416.0
3,She turned pregnant and he let her go! #shorts,2022-07-25T11:00:43Z,PT1M,415148935,3954689.0,3807.0
4,Tom&Jerry.. Real End Twist 😆#shorts,2023-06-13T13:40:23Z,PT1M,395791994,6965246.0,NaN
...,...,...,...,...,...,...
595,Sweet love story ❤️ #love #shorts #couplegoals...,2023-12-29T13:37:48Z,PT59S,339585,27195.0,73.0
596,Test an Verlobten😂 #funny #shorts,2024-01-05T16:15:35Z,PT1M1S,294921,23560.0,83.0
597,TESTE PARA SABER SE VOCÊ É ALIENÍGENA… 😰🛸👽 #sh...,2023-12-31T15:33:55Z,PT59S,297353,34489.0,731.0
598,Biasanya kebalikan malah🗿#shorts,2024-01-04T08:00:34Z,PT1M1S,289290,19309.0,239.0


In [21]:
# NULL ROWS FOR PARTICULAR COLUMNS
print("views:",data_set['view_count'].isnull().sum()," like_count:",data_set['like_count'].isnull().sum()," comment_count:",data_set['comment_count'].isnull().sum())

views: 0  like_count: 28  comment_count: 28


In [22]:
#Dropping rows having null values
data_set.dropna(inplace=True)

In [23]:
data_set.shape

(544, 6)

# EXPLORATORY DATA ANALYSIS

In [24]:
data_set = data_set[["title","view_count","like_count","comment_count","duration"]]